In [1]:
import tensorflow as tf
from tensorflow.contrib.rnn import BasicRNNCell

In [6]:
class RNNCell(object):
    def __init__(self, neuron_number):
        self.neuron_number=neuron_number
        
    def __call__(self, inputs, state):
        """the one step unrolling
        @param inputs: input from sequence [batch_size, feature_number]
        @param state: output of last step [batch_size, neuron_number]
        @return: the pair of (output, state), but they are the same in basic rnn
        """
        with tf.variable_scope("RNN_Cell"):
            W = tf.get_variable("W", shape=[inputs.get_shape()[1], self.neuron_number],
                                initializer=tf.random_normal_initializer(0, 0.1), dtype=tf.float32)
            U = tf.get_variable("U", shape=[state.get_shape()[1], self.neuron_number],
                                initializer=tf.random_normal_initializer(0, 0.1), dtype=tf.float32)
            b = tf.get_variable("b", shape=[self.neuron_number], dtype=tf.float32)
        outputs = tf.tanh(tf.matmul(inputs, W) + tf.matmul(state, U) + b)
        # Note that the this is row vector product here
        return outputs, outputs
        
    def zero_state(self):
        return tf.zeros(shape=[None, self.neuron_number])

In [3]:
cell = RNNCell(12)
cell.zero_state()
a = tf.placeholder(shape=(None, None, 30), dtype=tf.float32)
# a is input, [batch, time, neurons]

In [4]:
with tf.variable_scope("RNN") as rnn_sc:
    state = cell.zero_state()
    for i in range(10):
        output, state =cell(a[:,i,:], state)
        rnn_sc.reuse_variables()
        # to reuse the parameters, need to reset the reuse attribute of variable scope

## static unrolling
* the unrolling is done by the python code
  * which means that the total **steps of sequence** is fixed
  * padding technique could be applied here to mimic dynamic sequence
    * this is how tflearn do

In [5]:
output

<tf.Tensor 'RNN/Tanh_9:0' shape=(?, 12) dtype=float32>

In [6]:
state

<tf.Tensor 'RNN/Tanh_9:0' shape=(?, 12) dtype=float32>

In [7]:
def time_step(time, output_ta, state):
    output, state = cell(a[:, time, :], state)
    output_ta.write(time, output)
    return time+1, output, state

time_steps = tf.placeholder(shape=[], dtype=tf.int32)
time = tf.constant(0, dtype=tf.int32, name="time")
output_ta = tf.TensorArray(size=time_steps, dtype=tf.float32)
state = cell.zero_state()

_, final_output_ta, final_state = tf.while_loop(
    cond=lambda time, *_: time < time_steps,
    body=time_step,
    loop_vars=(time, output_ta, state))

ValueError: Variable RNN_Cell/W already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "c:\users\zhengyao\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\ops.py", line 1264, in __init__
    self._traceback = _extract_stack()
  File "c:\users\zhengyao\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\ops.py", line 2395, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "c:\users\zhengyao\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 763, in apply_op
    op_def=op_def)
